In [42]:
import math
import random
from collections import defaultdict

import numpy as np

In [43]:
# 逆関数法
def inv_func_method(F_inv):
    u = random.random()
    return F_inv(u)

def exp_dist_inv(u, lmd):
    if u < 0:
        raise ValueError
    elif 0 < u <= 1:
        return -1 / lmd * math.log(1 - u)
    else:
        raise ValueError

# 平均 mu の指数分布に従った乱数を返す
def exponential_dist(mu):
    return inv_func_method(lambda u: exp_dist_inv(u, 1 / mu))

In [44]:
class System_mm1k:
    def __init__(self, lmd, mu, K):
        self.lmd = lmd
        self.mu = mu
        self.K = K

        self.arrived = 0
        self.rejected = 0
        self.total_sys = 0
        self.total_waiting = 0
        self.busy_time = 0  # サーバーが忙しい時間の累積
        self.last_event_time = 0  # 前回イベントが発生した時刻

        self.t = 0  # 現在の時刻
        self.k = 0  # 現在のシステム内人数
        self.ea = 0

        self.next_arrival_time = self.next_arrival()  # 初回到着時間
        self.next_departure_time = float('inf')  # 初回サービス終了時間（初期は無限）
        
    def next_arrival(self):
        if self.lmd == 0:
            return 
        return exponential_dist(1 / self.lmd)

    def next_departure(self):
        return exponential_dist(1 / self.mu)

    def simulate_1event(self):
        if self.lmd == 0:
            self.last_event_time = float("inf")
            return
        if self.next_arrival_time < self.next_departure_time:  # 到着イベント
            self.t = self.next_arrival_time
            self.ea = self.t - self.last_event_time

            self.total_sys += self.ea * self.k
            self.total_waiting += self.ea * max(0, self.k - 1)
            
            self.arrived += 1
            if self.k > 0:  # サーバーが忙しい場合、前回イベントとの経過時間を加算
                self.busy_time += self.ea
            else:
                self.next_departure_time = self.t + self.next_departure()
            if self.k < self.K:
                self.k += 1
            else:
                self.rejected += 1
            self.next_arrival_time = self.t + self.next_arrival()  # 次の到着時間を計算
        else:  # サービス終了イベント
            self.t = self.next_departure_time
            self.ea = self.t - self.last_event_time

            self.total_sys += self.ea * self.k
            self.total_waiting += self.ea * max(0, self.k - 1)

            self.busy_time += self.ea
            self.k -= 1
            if self.k > 0:
                self.next_departure_time = self.t + self.next_departure()
            else:
                self.next_departure_time = float('inf')  # サーバーが空なら無限大

        self.last_event_time = self.t  # 現在の時刻を更新


In [45]:
def t(p, nu): # 自由度 nu, 累積確率 p
    if math.isclose(p, 0.025):
        if nu == 20:
            return 2.086
    else:
        raise ValueError

def calc_conf_itv(samples):
    num = len(samples)

    samples = np.array(samples)

    sample_mean = samples.mean()
    sample_var = samples.var()
    unbi_var = sample_var * num / (num - 1)

    conf_itv = 0.95
    p = (1 - conf_itv) / 2

    t_itv = t(p, num - 1) * np.sqrt(unbi_var / num)

    conf_low = sample_mean - t_itv
    conf_up = sample_mean + t_itv
    
    return sample_mean, conf_low, conf_up

# print(f"sample_mean: {sample_mean}")
# print(f"conf_itv: {conf_low} <= mu <= {conf_up}")

In [49]:
lmd = 8.0  # 到着率（λ）
mu = 10.0   # サービス率（μ）
K1, K2 = 6, 8
p = 0.3

packets = 1000000

results_K = []
results_sim = defaultdict(list)

for _ in range(21):
    system1 = System_mm1k(lmd * p, mu, K1)
    system2 = System_mm1k(lmd * (1 - p), mu, K2)
    


    while system1.arrived + system2.arrived < packets:
        if system1.last_event_time < system2.last_event_time:
            system1.simulate_1event()
        else:
            system2.simulate_1event()
            
    if system1.last_event_time == float("inf"):
        system1.last_event_time = 0
    if system2.last_event_time == float("inf"):
        system2.last_event_time = 0
        
        
    # System 1-specific metrics
    rej_rate1 = system1.rejected / system1.arrived
    E_L1 = system1.total_sys / system1.last_event_time
    E_Lq1 = system1.total_waiting / system1.last_event_time
    E_W1 = system1.total_sys / (system1.arrived - system1.rejected)
    E_Wq1 = system1.total_waiting / (system1.arrived - system1.rejected)
    util_rate1 = system1.busy_time / system1.last_event_time

    # System 2-specific metrics
    rej_rate2 = system2.rejected / system2.arrived
    E_L2 = system2.total_sys / system2.last_event_time
    E_Lq2 = system2.total_waiting / system2.last_event_time
    E_W2 = system2.total_sys / (system2.arrived - system2.rejected)
    E_Wq2 = system2.total_waiting / (system2.arrived - system2.rejected)
    util_rate2 = system2.busy_time / system2.last_event_time
    
    rej_rate = (system1.rejected + system2.rejected) / (system1.arrived + system2.arrived)
    E_L = (system1.total_sys * p + system2.total_sys * (1 - p)) / system1.last_event_time
    E_Lq = (system1.total_waiting * p + system2.total_waiting * (1 - p)) / (system1.last_event_time)
    E_W = (system1.total_sys + system2.total_sys) / (system1.arrived + system2.arrived - system1.rejected - system2.rejected)
    E_Wq = (system1.total_waiting + system2.total_waiting) / (system1.arrived + system2.arrived - system1.rejected - system2.rejected)
    util_rate = (system1.busy_time + system2.busy_time) / (system1.last_event_time + system2.last_event_time)

    arrived_rate = (system1.arrived) / (system1.arrived + system2.arrived)
    
# System 1-specific metrics
    results_sim["rej_rate1"].append(rej_rate1)
    results_sim["E_L1"].append(E_L1)
    results_sim["E_Lq1"].append(E_Lq1)
    results_sim["E_W1"].append(E_W1)
    results_sim["E_Wq1"].append(E_Wq1)
    results_sim["util_rate1"].append(util_rate1)

# System 2-specific metrics
    results_sim["rej_rate2"].append(rej_rate2)
    results_sim["E_L2"].append(E_L2)
    results_sim["E_Lq2"].append(E_Lq2)
    results_sim["E_W2"].append(E_W2)
    results_sim["E_Wq2"].append(E_Wq2)
    results_sim["util_rate2"].append(util_rate2)

# Combined metrics
    results_sim["rej_rate"].append(rej_rate)
    results_sim["E_L"].append(E_L)
    results_sim["E_Lq"].append(E_Lq)
    results_sim["E_W"].append(E_W)
    results_sim["E_Wq"].append(E_Wq)
    results_sim["util_rate"].append(util_rate)

# Arrival rate ratio
    results_sim["arrived_rate"].append(arrived_rate)

# print(results_sim)


In [48]:
for k, v in results_sim.items():
    sample_mean, conf_low, conf_up = calc_conf_itv(results_sim[k])

    print(f"{k:13}: sample_mean: {sample_mean:.6f} conf_itv: {conf_low:.6f} <= mu <= {conf_up:.6f}")

rej_rate1    : sample_mean: 0.000149 conf_itv: 0.000141 <= mu <= 0.000158
E_L1         : sample_mean: 0.315591 conf_itv: 0.315158 <= mu <= 0.316023
E_Lq1        : sample_mean: 0.075584 conf_itv: 0.075374 <= mu <= 0.075794
E_W1         : sample_mean: 0.131441 conf_itv: 0.131317 <= mu <= 0.131564
E_Wq1        : sample_mean: 0.031480 conf_itv: 0.031404 <= mu <= 0.031556
util_rate1   : sample_mean: 0.240007 conf_itv: 0.239752 <= mu <= 0.240261
rej_rate2    : sample_mean: 0.004276 conf_itv: 0.004211 <= mu <= 0.004341
E_L2         : sample_mean: 1.225065 conf_itv: 1.222810 <= mu <= 1.227320
E_Lq2        : sample_mean: 0.667083 conf_itv: 0.665145 <= mu <= 0.669022
E_W2         : sample_mean: 0.219641 conf_itv: 0.219289 <= mu <= 0.219993
E_Wq2        : sample_mean: 0.119601 conf_itv: 0.119279 <= mu <= 0.119922
util_rate2   : sample_mean: 0.557982 conf_itv: 0.557581 <= mu <= 0.558382
rej_rate     : sample_mean: 0.003038 conf_itv: 0.002993 <= mu <= 0.003082
E_L          : sample_mean: 1.540654 c